In [75]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import os
import matplotlib.pyplot as plt
import nltk
from collections import OrderedDict

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from collections import OrderedDict

[nltk_data] Downloading package punkt to /home/riya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/riya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/riya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [76]:
from keybert import KeyBERT

In [77]:
import gensim
from gensim.models import word2vec
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
import gensim.downloader as api
import matplotlib.pyplot as plt
import spacy
import string
from sklearn.metrics.pairwise import cosine_similarity

In [78]:
#list(gensim.downloader.info()['models'].keys())
#wv = api.load('glove-wiki-gigaword-300')
#wv.save('glove_wiki_vectors.kv')

In [145]:
revalive_path_vec = "../../glove_wiki_vectors.kv"
relative_path_sdg = "../../sdg_data.csv"
relative_path_attr = "../Dictionary/Attributes_2016-17.csv"
relative_string_map = "./Data_Indicator/String_map.csv"
relative_string_map_unroll = "./Data_Indicator/String_map_unroll.csv"
relative_semantic = "./Data_Indicator/Semantic_map.csv"
relative_semantic_unroll = "./Data_Indicator/Semantic_map_unroll.csv"
relative_sem_threshold = "./Data_Indicator/Semantic_threshold_map.csv"
relative_sem_threshold_unroll = "./Data_Indicator/Semantic_threshold_map_unroll.csv"

# Get the absolute path by joining the current directory with the relative path
absolute_path_vec = os.path.normpath(os.path.join(os.getcwd(), revalive_path_vec))
absolute_path_sdg = os.path.normpath(os.path.join(os.getcwd(), relative_path_sdg))
absolute_path_attr = os.path.normpath(os.path.join(os.getcwd(), relative_path_attr))
absolute_path_string = os.path.normpath(os.path.join(os.getcwd(), relative_string_map))
absolute_path_string_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_string_map_unroll))
absolute_path_semantic = os.path.normpath(os.path.join(os.getcwd(), relative_semantic))
absolute_path_semantic_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_semantic_unroll))
absolute_path_sem_threshold =os.path.normpath( os.path.join(os.getcwd(), relative_sem_threshold))
absolute_path_sem_threshold_unroll = os.path.normpath(os.path.join(os.getcwd(), relative_sem_threshold_unroll))

In [80]:
wv = KeyedVectors.load(absolute_path_vec)

In [81]:
kw_model = KeyBERT(model=wv)

In [82]:
def replace_words(df, column_list, replace_dict):
    for col in column_list:
        # Apply the replace function to the specified column using the replace_dict
        df[col] = df[col].replace(replace_dict, regex=True)
    return df

# Define the replacement dictionary
replace_dict = {
    'B.p.l.': 'below poverty line',
    'A.p.l.': 'above poverty line',
    'Bpl': 'below poverty line',
}

In [83]:
IGNORE_WORDS = ['proportion', 'whose', 'number', 'per', 'within', 'no', 'under', 'volume', 'use', 'type', 'percentage', 'total', 'data', 'year', 'due', 'rate', 'hects', 'hect', 'noof', 'years']

In [84]:
# Creating our tokenizer function
def cleaning(sentence):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    wn = nltk.WordNetLemmatizer()
    ignore_word = []
    cleaned = []
    for i in range(len(sentence)):
        letters_1 = re.sub('[^a-zA-Z.]', ' ', sentence[i])
        letters = re.sub(r'\.', '', letters_1)
        letters = letters.lower().split() 
        letters = [wn.lemmatize(w) for w in letters if w not in stopwords ]
        IGNORE_WORDS_LEM = [wn.lemmatize(w).lower() for w in IGNORE_WORDS]
        letters = [w for w in letters if w not in IGNORE_WORDS_LEM]
        #letters_list = list(set(letters))
        letters_list = list(OrderedDict.fromkeys(letters))   
        cleaned.append(letters_list)

    return cleaned

In [85]:
# Creating our tokenizer function
def concat_cleaning(sentence):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    wn = nltk.WordNetLemmatizer()

    cleaned = []
    for i in range(len(sentence)):
        letters_1 = re.sub('[^a-zA-Z.]', ' ', sentence[i])
        letters = re.sub(r'\.', '', letters_1)
        letters = letters.lower().split() 
        #letters = [wn.lemmatize(w) for w in letters if w not in stopwords ]
        #IGNORE_WORDS_LEM = [wn.lemmatize(w).lower() for w in IGNORE_WORDS]
        #letters = [w for w in letters if w not in IGNORE_WORDS_LEM]
        letters = ' '.join(letters)
        cleaned.append(letters)

    return cleaned

In [86]:
def process_keyword(data):
    result = []
    for sublist in data:
        inner_list = []
        for word, _ in sublist:
            # Split the word if it contains a space and reverse the order of words
            words = word.split()
            # Filter out numeric values
            words = [w for w in words if not w.isdigit()]
            if words:  # Exclude empty lists
                inner_list.append(words)
        inner_list.reverse()
        result.append(inner_list)
    return result

In [87]:
def add_elements(row):
    if not row['keywords']:  # Check if col2 is empty
        row['keywords'].extend([[elem] for elem in row['new_indicators']])  # Add elements from col1 to col2 as lists
    return row

In [88]:
def concatenate_table_name(df):
    for index, row in df.iterrows():
        if ',' not in row['Description']:
            df.at[index, 'Description'] += ', ' + row['Table_name']
        elif row['Description'].count(',') == 1:
            parts = row['Description'].split(',')
            if len(parts) == 2:
                part1, part2 = parts
                if re.match(r'^\w+$', part1.strip()) or re.match(r'^\w+$', part2.strip()):
                    df.at[index, 'Description'] += ', ' + row['Table_name']
    return df


In [89]:
def concatenate_all(df):
    for index, row in df.iterrows():
        df.at[index, 'Description'] += ', ' + row['Table_name'] + ', ' + row['Chapter_name'] 
    return df

In [90]:
missing_words = []
miss_added = []
def sent_vec(row):
    sent = row['new_description']
    cap_tab = row['Chapter_name'] + ", " + row['Table_name']
    vector_size = wv.vector_size
    vectors = []
    miss_w = []
    for w in sent:
        if w in wv:
            wv_res = np.zeros(vector_size)
            wv_res = wv[w]
            vectors.append(wv_res)
        else:
            missing_words.append(("KAG", w))
            miss_w.append(w)
    if len(miss_w)>0:
        vectors.append(missin_word_vec(cap_tab))
        miss_added.append(miss_w)
    vec_arr = np.array(vectors)
    len_vec = len(vectors)
    return vec_arr

In [91]:
def missin_word_vec(sent):
    vector_size = wv.vector_size
    sent_list = cleaning([sent])
    wv_res = np.zeros(vector_size)
    ctr = 0
    for w in sent_list[0]:
        if w in wv:
            wv_res += wv[w]
            ctr += 1
    if ctr>0:
        wv_res = wv_res/ctr
    else:
        wv_res = np.random.rand(vector_size)
        print('yes')
    return wv_res

In [92]:
def sent_vec_avg(sent):
    vector_size = wv.vector_size
    missing_words = []
    vectors = []
    for w in sent:
        ctr = 0
        wv_res = np.zeros(vector_size)
        for word in  w:
            if word in wv:
                wv_res += wv[word]
                ctr += 1
            else:
                missing_words.append(word)
        if ctr>0:
            wv_res = wv_res/ctr
        vectors.append(wv_res)
    vec_arr = np.array(vectors)
    len_vec = len(vectors)
    return vec_arr

In [93]:
def count_common_words(row, df_other):
    set_1 = set(row)
    common_words = []
    for _, other_row in df_other.iterrows():
        set_2 = set(other_row['new_description'])
        common_words.append(len(set_1.intersection(set_2)))
    return common_words

In [94]:
def find_similarity_string(similarity_matrix, threshold, chapter_df, sdg_df):
    top_n_similarities = []
    top_n_attr = []
    top_n_chapter_id = []
    top_n_chapter_name = []
    top_n_table_id = []
    top_n_table_name = []
    top_n_count = []
    top_n_description = []
    
    for index, row in similarity_matrix.iterrows():
        # Filter indices based on threshold value
        top_n_indices = [i for i, value in enumerate(row) if value >= threshold]
        
        # Sort indices based on similarity values
        top_n_indices = sorted(top_n_indices, key=lambda i: row.iloc[i], reverse=True)
        
        # Get top n values, chapters, and chapter names
        values = [row.iloc[i] for i in top_n_indices]
        attr = [similarity_matrix.columns[i] for i in top_n_indices]
        attr_des = [chapter_df['Description'].loc[i] for i in top_n_indices]
        chapter_id = [chapter_df['Chapter_id'].loc[i] for i in top_n_indices]
        chapter_name = [chapter_df['Chapter_name'].loc[i] for i in top_n_indices]   
        table_id = [chapter_df['Table_id'].loc[i] for i in top_n_indices]
        table_name = [chapter_df['Table_name'].loc[i] for i in top_n_indices]   


        # Append to respective lists
        top_n_count.append(len(attr))
        top_n_similarities.append(values)
        top_n_attr.append(attr)
        top_n_description.append(attr_des)
        top_n_chapter_id.append(chapter_id)
        top_n_chapter_name.append(chapter_name)
        top_n_table_id.append(table_id)
        top_n_table_name.append(table_name)
    
    # Assign lists to the SDG DataFrame
    sdg_df['top_n_count'] = top_n_count
    sdg_df['top_n_attr'] = top_n_attr
    sdg_df['top_n_table_id'] = top_n_table_id
    sdg_df['top_n_table_name'] = top_n_table_name
    sdg_df['top_n_chapter_id'] = top_n_chapter_id
    sdg_df['top_n_chapter_name'] = top_n_chapter_name
    sdg_df['top_n_similarities'] = top_n_similarities
    sdg_df['top_n_description'] = top_n_description

    return sdg_df


In [95]:
def process_result_string(result_sdg, sdg_df):
    rows = []
    
    for _, row in result_sdg.iterrows():
        attr_id = row['top_n_attr']
        chapter_id = row['top_n_chapter_id']
        table_id = row['top_n_table_id']
        values = row['top_n_similarities']
        attr_desc = row['top_n_description']
        chapter_name = row['top_n_chapter_name']
        table_name = row['top_n_table_name']
        goal_no = row['Goal No.']
        target_id = row['Target_id']
        indicator_id = row['Indicator_id']
        
        for attr, chap, tab, val, desc, chap_name, tab_name in zip(
            attr_id, chapter_id, table_id, values, attr_desc, chapter_name, table_name
        ):
            rows.append({
                'Attr_id': attr,
                'Chapter_id': chap,
                'Chapter_name': chap_name,
                'Table_id': tab,
                'Table_name': tab_name,
                'Description': desc,
                'top_n_goal_id': goal_no,
                'top_n_target_id': target_id,
                'top_n_indicator_id': indicator_id,
                'top_n_similarities': val
            })

    all_goals = pd.DataFrame(rows)
    
    goal_name = dict(zip(sdg_df['Goal No.'], sdg_df['Goal']))
    target_name = dict(zip(sdg_df['Target_id'], sdg_df['Targets']))
    indicator_name = dict(zip(sdg_df['Indicator_id'], sdg_df['Tentative Indicators']))
    
    all_goals['Goal'] = all_goals['top_n_goal_id'].map(goal_name.get)
    all_goals['Targets'] = all_goals['top_n_target_id'].map(target_name.get)
    all_goals['Tentative Indicators'] = all_goals['top_n_indicator_id'].map(indicator_name.get)
    all_goals = all_goals.loc[:,['Attr_id','Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description', 'Goal', 'Targets', 'Tentative Indicators', 'top_n_goal_id', 'top_n_target_id', 'top_n_indicator_id', 'top_n_similarities']]
    return all_goals


In [96]:
def find_similarity_semantic(sdg_df, att_df, threshold):
    ind_list = sdg_df['vec'].tolist()
    att_list = att_df['vec'].tolist()
    min_similarity_values = []
    top_indices_values = []
    for ind in range(len(ind_list)):
        top_indices_list = []
        top_sim_list = []
        top_min_list = []
        for i in ind_list[ind]:
            max_list = []
            for att in range(len(att_list)):
                similarity_matrix = cosine_similarity([i], att_list[att])
                max_similarity = np.max(similarity_matrix)
                max_list.append(max_similarity)
            # Get the indices of the top 10 max values
            #top_indices = [idx for idx, sim in enumerate(max_list) if sim < 1]
            top_indices = sorted(range(len(max_list)), key=lambda i: max_list[i], reverse=True)[:threshold]
            top_indices_list.append(top_indices)
            top_sim = [max_list[i] for i in top_indices]
            top_sim_list.append(top_sim)
            top_min_list.append(min(top_sim))
            
        # Merge the inner lists of l1 and l2
        merged_index = [item for sublist in top_indices_list for item in sublist]
        merged_sim = [item for sublist in top_sim_list for item in sublist]
        
        # Sort l1 based on the values in l2
        sorted_indices = sorted(range(len(merged_sim)), key=lambda i: merged_sim[i], reverse=True)
        sorted_index = [merged_index[i] for i in sorted_indices]
        final_index_list = list(OrderedDict.fromkeys(sorted_index))
            
        #top_indices_set = list(OrderedDict.fromkeys(top_indices_list))
        top_indices_values.append(final_index_list)
        min_similarity_values.append(top_min_list)
    return min_similarity_values, top_indices_values


In [97]:
#indicator words to attribute complete name
def find_similarity_semantic_threshold(sdg_df, att_df, diff):
    ind_list = sdg_df['vec'].tolist()
    att_list = att_df['vec'].tolist()
    min_list = sdg_df['min_values'].tolist()
    top_indices_values = []
    for ind in range(len(ind_list)):
        top_indices_list = []
        top_sim_list = []
        min_values = min_list[ind]
        for index, i in enumerate(ind_list[ind]):
            max_list = []
            threshold = min_values[index] - diff
            if(threshold <= 0):
                continue
            for att in range(len(att_list)):
                similarity_matrix = cosine_similarity([i], att_list[att])
                max_value = np.max(similarity_matrix)
                max_list.append((max_value, att))
            
            # Get the indices of values strictly greater than threshold
            top_indices = [idx for val, idx in max_list if val > threshold]
            # Sort top_indices based on the corresponding values in max_list
            top_indices.sort(key=lambda x: next((val for val, idx in max_list if idx == x), 0), reverse=True)
            top_indices_list.append(top_indices)
            top_sim = [max_list[i][0] for i in top_indices]
            top_sim_list.append(top_sim)
        # Merge the inner lists of l1 and l2
        merged_index = [item for sublist in top_indices_list for item in sublist]
        merged_sim = [item for sublist in top_sim_list for item in sublist]
        
        # Sort l1 based on the values in l2
        sorted_indices = sorted(range(len(merged_sim)), key=lambda i: merged_sim[i], reverse=True)
        sorted_index = [merged_index[i] for i in sorted_indices]
        final_index_list = list(OrderedDict.fromkeys(sorted_index))
        
        #top_indices_set = list(OrderedDict.fromkeys(top_indices_list))
        top_indices_values.append(final_index_list)
        
    return top_indices_values

In [98]:
def finding_attr_details(sdg_df, att_df):
    result_list = sdg_df['top_n_index'].tolist()
    top_n_count = []
    top_n_attr = []
    top_n_chapter_id = []
    top_n_table_id = []
    top_n_description = []
    top_n_chapter_name = []
    top_n_table_name = []
    for ll in result_list:
        top_n_count.append(len(ll))
        top_n_attr.append([att_df.loc[idx, 'Attr_id'] for idx in ll])
        top_n_chapter_id.append([att_df.loc[idx, 'Chapter_id'] for idx in ll])
        top_n_table_id.append([att_df.loc[idx, 'Table_id'] for idx in ll])
        top_n_description.append([att_df.loc[idx, 'Description'] for idx in ll])
        top_n_chapter_name.append([att_df.loc[idx, 'Chapter_name'] for idx in ll])
        top_n_table_name.append([att_df.loc[idx, 'Table_name'] for idx in ll])
    sdg_df['top_n_count'] = top_n_count
    sdg_df['top_n_attr'] = top_n_attr
    sdg_df['top_n_chapter_id'] = top_n_chapter_id
    sdg_df['top_n_table_id'] = top_n_table_id
    sdg_df['top_n_description'] = top_n_description
    sdg_df['top_n_chapter_name'] = top_n_chapter_name
    sdg_df['top_n_table_name'] = top_n_table_name
    return sdg_df

In [99]:
def process_result_semantic(result_sdg, sdg_df):
    rows = []
    
    for _, row in result_sdg.iterrows():
        attr_id = row['top_n_attr']
        chapter_id = row['top_n_chapter_id']
        table_id = row['top_n_table_id']
        attr_desc = row['top_n_description']
        chapter_name = row['top_n_chapter_name']
        table_name = row['top_n_table_name']
        goal_no = row['Goal No.']
        target_id = row['Target_id']
        indicator_id = row['Indicator_id']
        
        for attr, chap, tab, desc, chap_name, tab_name in zip(
            attr_id, chapter_id, table_id, attr_desc, chapter_name, table_name
        ):
            rows.append({
                'Attr_id': attr,
                'Chapter_id': chap,
                'Chapter_name': chap_name,
                'Table_id': tab,
                'Table_name': tab_name,
                'Description': desc,
                'top_n_goal_id': goal_no,
                'top_n_target_id': target_id,
                'top_n_indicator_id': indicator_id
            })

    all_goals = pd.DataFrame(rows)
    
    goal_name = dict(zip(sdg_df['Goal No.'], sdg_df['Goal']))
    target_name = dict(zip(sdg_df['Target_id'], sdg_df['Targets']))
    indicator_name = dict(zip(sdg_df['Indicator_id'], sdg_df['Tentative Indicators']))
    
    all_goals['Goal'] = all_goals['top_n_goal_id'].map(goal_name.get)
    all_goals['Targets'] = all_goals['top_n_target_id'].map(target_name.get)
    all_goals['Tentative Indicators'] = all_goals['top_n_indicator_id'].map(indicator_name.get)
    all_goals = all_goals.loc[:,['Attr_id','Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description', 'Goal', 'Targets', 'Tentative Indicators', 'top_n_goal_id', 'top_n_target_id', 'top_n_indicator_id']]
    return all_goals


In [100]:
sdg = pd.read_csv(absolute_path_sdg, sep=",", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str})

In [101]:
sdg

,Goal No.,Goal,Nodal Department,Targets,Other Related Major Departments,Tentative Indicators,Target_id,Indicator_id
0,1,End poverty in all its forms everywhere,Rural Development,"By 2030, eradicate extreme poverty for all peo...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population below the interna...,1.1,1.1.1
1,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living below the ...,1.2,1.2.1
2,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...","Proportion of men, women and children of all a...",1.2,1.2.2
3,1,End poverty in all its forms everywhere,Rural Development,Implement nationally appropriate social protec...,"Urban Development, Agriculture, Horticulture, ...",Percentage of the population covered by social...,1.3,1.3.1
4,1,End poverty in all its forms everywhere,Rural Development,"By 2030, ensure that all men and women, in par...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living in househo...,1.4,1.4.1
...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,Finance,"Promote the development, transfer, disseminati...",Environment and Scientific Technology.,Total amount of approved funding to promote th...,16.4,16.4.1
175,16,Strengthen the means of implementation and rev...,Finance,Fully operationalize the technology bank and s...,"Environment and Scientific Technology, Inform...",Proportion of individuals using the Internet.,16.5,16.5.1
176,17,"Data, monitoring and accountability",Finance,"By 2020, enhance capacity-building support to ...","Planning, Finance, Economic and Statistics.",Proportion of sustainable development indicato...,17.1,17.1.1
177,17,"Data, monitoring and accountability",Finance,"By 2030, build on existing initiatives to deve...","Planning, Finance, Economic and Statistics.",Dollar value of all resources made available t...,17.2,17.2.1


In [102]:
sdg.columns

Index(['Goal No.', 'Goal', 'Nodal Department', 'Targets',
       'Other Related Major Departments', 'Tentative Indicators', 'Target_id',
       'Indicator_id'],
      dtype='object')

In [103]:
sdg.drop(['Nodal Department','Other Related Major Departments'],axis=1,inplace=True)

In [104]:
# Remove duplicate rows
sdg = sdg.drop_duplicates()

In [105]:
sdg.reset_index(inplace = True, drop = True)

In [106]:
sdg

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1
...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1
175,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,Proportion of individuals using the Internet.,16.5,16.5.1
176,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",Proportion of sustainable development indicato...,17.1,17.1.1
177,17,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Dollar value of all resources made available t...,17.2,17.2.1


In [107]:
sdg_new = sdg.copy()

In [108]:
texto_data=sdg_new['Tentative Indicators'].tolist()
process_text=cleaning(texto_data)

cleaned_sent = pd.DataFrame({'new_indicators': process_text})
sdg_new['new_indicators']=cleaned_sent

In [109]:
texto_data0=sdg_new['Tentative Indicators'].tolist()
process_text0=concat_cleaning(texto_data0)

cleaned_sent0 = pd.DataFrame({'new_indicators_str': process_text0})
sdg_new['new_indicators_str']=cleaned_sent0

In [110]:
sdg_new

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...
...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...
175,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,Proportion of individuals using the Internet.,16.5,16.5.1,"[individual, using, internet]",proportion of individuals using the internet
176,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",Proportion of sustainable development indicato...,17.1,17.1.1,"[sustainable, development, indicator, produced...",proportion of sustainable development indicato...
177,17,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Dollar value of all resources made available t...,17.2,17.2.1,"[dollar, value, resource, made, available, str...",dollar value of all resources made available t...


In [111]:
data = pd.read_csv(absolute_path_attr, sep=";", dtype={'Attr_id': str, 'Chapter_id': str, 'Table_id':str})

In [112]:
# Replace words in the DataFrame using the dictionary
data = replace_words(data, ['Chapter_name', 'Table_name', 'Description'], replace_dict)

In [113]:
data_new = concatenate_all(data)
data_new

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description
0,3.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Nada Offices, Nada Offices Village Accountant ..."
1,4.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Va Circles, Nada Offices Village Accountant Ci..."
2,5.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Hoblies, Nada Offices Village Accountant Circl..."
3,6.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Grama Panchayaths, Nada Offices Village Accoun..."
4,7.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"No.of Taluks, Nada Offices Village Accountant ..."
...,...,...,...,...,...,...
1162,1169.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Total,Number,Total Agrl. Land Holder Total, Ag..."
1163,1170.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Male,Area,Total Agrl. Land Holder Total, Agric..."
1164,1171.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Female,Area,Total Agrl. Land Holder Total, Agr..."
1165,1172.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Institutions,Area,Total Agrl. Land Holder Tota..."


In [114]:
# Create a new DataFrame with selected columns
new_df = data_new.copy()
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description
0,3.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Nada Offices, Nada Offices Village Accountant ..."
1,4.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Va Circles, Nada Offices Village Accountant Ci..."
2,5.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Hoblies, Nada Offices Village Accountant Circl..."
3,6.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Grama Panchayaths, Nada Offices Village Accoun..."
4,7.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"No.of Taluks, Nada Offices Village Accountant ..."
...,...,...,...,...,...,...
1162,1169.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Total,Number,Total Agrl. Land Holder Total, Ag..."
1163,1170.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Male,Area,Total Agrl. Land Holder Total, Agric..."
1164,1171.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Female,Area,Total Agrl. Land Holder Total, Agr..."
1165,1172.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Institutions,Area,Total Agrl. Land Holder Tota..."


In [115]:
texto_data1=new_df['Description'].tolist()
process_text1=cleaning(texto_data1)

cleaned_sent1 = pd.DataFrame({'new_description': process_text1})
new_df['new_description']=cleaned_sent1

In [116]:
texto_data11=new_df['Description'].tolist()
process_text11=concat_cleaning(texto_data11)

cleaned_sent11 = pd.DataFrame({'new_description_str': process_text11})
new_df['new_description_str']=cleaned_sent11

In [117]:
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,new_description,new_description_str
0,3.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Nada Offices, Nada Offices Village Accountant ...","[nada, office, village, accountant, circle, ho...",nada offices nada offices village accountant c...
1,4.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Va Circles, Nada Offices Village Accountant Ci...","[va, circle, nada, office, village, accountant...",va circles nada offices village accountant cir...
2,5.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Hoblies, Nada Offices Village Accountant Circl...","[hoblies, nada, office, village, accountant, c...",hoblies nada offices village accountant circle...
3,6.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Grama Panchayaths, Nada Offices Village Accoun...","[grama, panchayaths, nada, office, village, ac...",grama panchayaths nada offices village account...
4,7.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"No.of Taluks, Nada Offices Village Accountant ...","[taluks, nada, office, village, accountant, ci...",noof taluks nada offices village accountant ci...
...,...,...,...,...,...,...,...,...
1162,1169.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Total,Number,Total Agrl. Land Holder Total, Ag...","[agrl, land, holder, agricultural, holding, ar...",total number total agrl land holder total agri...
1163,1170.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Male,Area,Total Agrl. Land Holder Total, Agric...","[male, area, agrl, land, holder, agricultural,...",male area total agrl land holder total agricul...
1164,1171.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Female,Area,Total Agrl. Land Holder Total, Agr...","[female, area, agrl, land, holder, agricultura...",female area total agrl land holder total agric...
1165,1172.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Institutions,Area,Total Agrl. Land Holder Tota...","[institution, area, agrl, land, holder, agricu...",institutions area total agrl land holder total...


String Matching

In [118]:
common_word_matrix = sdg_new['new_indicators'].apply(count_common_words, args=(new_df,))
common_word_matrix

0      [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...
1      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                             ...                        
174    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
175    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
176    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
177    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
178    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: new_indicators, Length: 179, dtype: object

In [119]:
attr_id = new_df['Attr_id'].tolist()
ind_id = sdg_new['Indicator_id'].tolist()

In [120]:
# Determine the maximum number of words in any list
max_words = max(common_word_matrix, key=lambda x: len(x))

# Create DataFrame with columns Word_1, Word_2, ..., Word_n
common_word_df = pd.DataFrame(common_word_matrix.tolist(), columns=attr_id, index=ind_id)

In [121]:
common_word_df

,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,...,1164.0,1165.0,1166.0,1167.0,1168.0,1169.0,1170.0,1171.0,1172.0,1173.0
1.1.1,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1.2.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.2.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.3.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1.4.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16.4.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16.5.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17.1.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17.2.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
# Example usage:
threshold_value = 1  # Define the threshold value

# Call the function to find chapter_ids with similarity greater than the threshold
sdg_string_map = find_similarity_string(common_word_df, threshold_value, new_df, sdg_new.copy())


In [123]:
sdg_string_map

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,top_n_count,top_n_attr,top_n_table_id,top_n_table_name,top_n_chapter_id,top_n_chapter_name,top_n_similarities,top_n_description
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...,197,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[2.11, 2.11, 2.11, 2.11, 2.11, 2.11, 2.11, 2.1...",[Districtwise Population By Age Groups And Sex...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, ...","[Area and Population, Area and Population, Are...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, ...","[Male,0-14, Districtwise Population By Age Gro..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...,130,"[120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126...","[2.11, 2.11, 2.11, 2.11, 2.11, 2.11, 2.11, 2.1...",[Districtwise Population By Age Groups And Sex...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, ...","[Area and Population, Area and Population, Are...","[4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 2, 2, 2, 2, 2, ...","[Male,0-14, Districtwise Population By Age Gro..."
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...,94,"[575.0, 576.0, 577.0, 578.0, 579.0, 580.0, 839...","[10.8, 10.8, 10.8, 10.8, 10.8, 10.8, 13.1, 13....",[No .of Children in the age group 6 to 14 year...,"[10, 10, 10, 10, 10, 10, 13, 13, 13, 13, 13, 1...","[Education, Education, Education, Education, E...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[Boys,No.of Children Enrolment age group betwe..."
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...,263,"[76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83....","[2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, 2.4, ...",[Child Population 0-6 and proportion by reside...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[Area and Population, Area and Population, Are...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[Total,Child Population 0-6 Years 2011, Child ..."
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...,187,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.2, 2.2, 2.2, ...",[Population and percentage share to total Popu...,"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[Area and Population, Area and Population, Are...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[Geographical Area Sq.Kms, Population and perc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...,56,"[839.0, 840.0, 841.0, 842.0, 843.0, 844.0, 845...","[13.1, 13.1, 13.1, 13.1, 13.1, 13.2, 13.2, 13....",[Stree Shakti Self Help Groups and Members Yea...,"[13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 1...","[Women & Child Development, Women & Child Deve...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...

In [124]:
string_all_goals = process_result_string(sdg_string_map, sdg_new)

In [125]:
sdg_string_map.to_csv(absolute_path_string, sep=";", index=False)

In [126]:
string_all_goals.to_csv(absolute_path_string_unroll, sep=";", index=False)

Semantic Mapping

In [127]:
sdg_sem = sdg_new.copy()

In [128]:
word_count = sdg_sem['new_indicators'].apply(lambda x: len(x))
word_count

0      14
1       9
2      10
3      23
4       6
       ..
174    11
175     3
176    15
177     8
178     3
Name: new_indicators, Length: 179, dtype: int64

In [129]:
from collections import Counter
w_c = Counter(word_count)
w_c.most_common()

[(5, 21),
 (6, 18),
 (7, 17),
 (4, 17),
 (9, 16),
 (10, 15),
 (8, 14),
 (3, 12),
 (12, 8),
 (11, 6),
 (16, 5),
 (2, 5),
 (14, 4),
 (13, 4),
 (15, 4),
 (20, 2),
 (26, 2),
 (17, 2),
 (23, 1),
 (24, 1),
 (22, 1),
 (29, 1),
 (27, 1),
 (21, 1),
 (28, 1)]

In [130]:
w_c.most_common(10)

[(5, 21),
 (6, 18),
 (7, 17),
 (4, 17),
 (9, 16),
 (10, 15),
 (8, 14),
 (3, 12),
 (12, 8),
 (11, 6)]

In [131]:
np.mean(word_count)

8.726256983240223

In [132]:
median_indicator = np.median(word_count)
print(median_indicator)

7.0


In [133]:
from statistics import mode
mode_indicator = mode(word_count)
mode_indicator

5

In [134]:
stopwords_set = list(set(nltk.corpus.stopwords.words('english')))
stopwords_set.extend(IGNORE_WORDS)
stopwords_set = list(set(stopwords_set))
stopwords_set

['am',
 'than',
 "you've",
 'because',
 've',
 'him',
 't',
 'type',
 'aren',
 'his',
 'their',
 'her',
 'then',
 'hasn',
 'some',
 'hects',
 'o',
 'after',
 'whose',
 'total',
 'out',
 's',
 'do',
 'under',
 'but',
 "doesn't",
 'is',
 'your',
 "you'll",
 "weren't",
 'doesn',
 "shan't",
 'has',
 'number',
 'each',
 'ain',
 'where',
 'before',
 'what',
 'if',
 'from',
 'this',
 'doing',
 'up',
 "won't",
 'should',
 "should've",
 'shan',
 'it',
 'other',
 "you'd",
 'wasn',
 'volume',
 'further',
 'all',
 'against',
 'year',
 'its',
 'll',
 "it's",
 'being',
 'between',
 'haven',
 'here',
 'to',
 'mustn',
 'me',
 'won',
 'data',
 'hect',
 'at',
 'due',
 'years',
 'were',
 'did',
 'while',
 "you're",
 'noof',
 "isn't",
 'a',
 'off',
 'they',
 'mightn',
 'of',
 'he',
 'herself',
 'been',
 'having',
 'too',
 "needn't",
 'with',
 'about',
 "mightn't",
 'by',
 "didn't",
 'yourself',
 "couldn't",
 "wasn't",
 'yourselves',
 'isn',
 'm',
 "wouldn't",
 'was',
 'there',
 'who',
 'very',
 'such',
 '

In [135]:
#using 'new_indicators_str', takin top_n=6 as median = 8 and nr_candidates=10 as mean = 9.2
sdg_sem['keywords']=kw_model.extract_keywords(sdg_sem['new_indicators_str'].tolist(), keyphrase_ngram_range=(1,2), stop_words = stopwords_set, use_maxsum=True, nr_candidates=10, top_n=6)

In [136]:
sdg_sem['keywords'] = process_keyword(sdg_sem['keywords'].tolist())
sdg_sem

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,keywords
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...,"[[poverty, line], [population], [age, group], ..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...,"[[living, national], [poverty, line], [age, gr..."
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...,"[[according, national], [children, ages], [liv..."
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...,"[[age, persons], [children], [population, cove..."
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...,"[[basic, services], [population], [living], [a..."
...,...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...,"[[development], [sound, technologies], [promot..."
175,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,Proportion of individuals using the Internet.,16.5,16.5.1,"[individual, using, internet]",proportion of individuals using the internet,[]
176,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",Proportion of sustainable development indicato...,17.1,17.1.1,"[sustainable, development, indicator, produced...",proportion of sustainable development indicato...,"[[official, statistics], [level], [target, acc..."
177,17,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Dollar value of all resources made available t...,17.2,17.2.1,"[dollar, value, resource, made, available, str...",dollar value of all resources made available t...,"[[dollar, value], [statistical, capacity], [va..."


In [137]:
# Apply the function to each row of the DataFrame
sdg_sem = sdg_sem.apply(add_elements, axis=1)
sdg_sem

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,keywords
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...,"[[poverty, line], [population], [age, group], ..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...,"[[living, national], [poverty, line], [age, gr..."
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...,"[[according, national], [children, ages], [liv..."
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...,"[[age, persons], [children], [population, cove..."
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...,"[[basic, services], [population], [living], [a..."
...,...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...,"[[development], [sound, technologies], [promot..."
175,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,Proportion of individuals using the Internet.,16.5,16.5.1,"[individual, using, internet]",proportion of individuals using the internet,"[[individual], [using], [internet]]"
176,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",Proportion of sustainable development indicato...,17.1,17.1.1,"[sustainable, development, indicator, produced...",proportion of sustainable development indicato...,"[[official, statistics], [level], [target, acc..."
177,17,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Dollar value of all resources made available t...,17.2,17.2.1,"[dollar, value, resource, made, available, str...",dollar value of all resources made available t...,"[[dollar, value], [statistical, capacity], [va..."


In [138]:
sdg_sem['vec'] = sdg_sem['keywords'].apply(sent_vec_avg)
sdg_sem

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,keywords,vec
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...,"[[poverty, line], [population], [age, group], ...","[[-0.07521600276231766, 0.474590003490448, 0.2..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...,"[[living, national], [poverty, line], [age, gr...","[[-0.21916300430893898, 0.3655640622600913, 0...."
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...,"[[according, national], [children, ages], [liv...","[[-0.17308800294995308, 0.43849001079797745, 0..."
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...,"[[age, persons], [children], [population, cove...","[[-0.5623050034046173, 0.039556750562042, 0.20..."
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...,"[[basic, services], [population], [living], [a...","[[-0.6532500088214874, -0.4114450067281723, -0..."
...,...,...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...,"[[development], [sound, technologies], [promot...","[[-0.10136999934911728, -0.27173998951911926, ..."
175,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,Proportion of individuals using the Internet.,16.5,16.5.1,"[individual, using, internet]",proportion of individuals using the internet,"[[individual], [using], [internet]]","[[-0.5918700098991394, 0.49957001209259033, -0..."
176,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",Proportion of sustainable development indicato...,17.1,17.1.1,"[sustainable, development, indicator, produced...",proportion of sustainable development indicato...,"[[official, statistics], [level], [target, acc...","[[-0.10437500174157321, 0.33179498463869095, 0..."
177,17,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Dollar value of all resources made available t...,17.2,17.2.1,"[dollar, value, resource, made, available, str...",dollar value of all resources made available t...,"[[dollar, value], [statistical, capacity], [va...","[[-0.328249990940094, 0.2354124989360571, -0.1..."


In [139]:
new_df['vec'] = new_df.apply(sent_vec, axis=1)

In [140]:
new_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,new_description,new_description_str,vec
0,3.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Nada Offices, Nada Offices Village Accountant ...","[nada, office, village, accountant, circle, ho...",nada offices nada offices village accountant c...,"[[-0.07122399657964706, -0.3697200119495392, -..."
1,4.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Va Circles, Nada Offices Village Accountant Ci...","[va, circle, nada, office, village, accountant...",va circles nada offices village accountant cir...,"[[-0.1175599992275238, -0.5297099947929382, 0...."
2,5.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Hoblies, Nada Offices Village Accountant Circl...","[hoblies, nada, office, village, accountant, c...",hoblies nada offices village accountant circle...,"[[-0.07122399657964706, -0.3697200119495392, -..."
3,6.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"Grama Panchayaths, Nada Offices Village Accoun...","[grama, panchayaths, nada, office, village, ac...",grama panchayaths nada offices village account...,"[[0.09846500307321548, -0.4200499951839447, -0..."
4,7.0,1,General Information,1.1,Nada Offices Village Accountant Circles Hoblie...,"No.of Taluks, Nada Offices Village Accountant ...","[taluks, nada, office, village, accountant, ci...",noof taluks nada offices village accountant ci...,"[[-0.18463000655174255, -1.0946999788284302, -..."
...,...,...,...,...,...,...,...,...,...
1162,1169.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Total,Number,Total Agrl. Land Holder Total, Ag...","[agrl, land, holder, agricultural, holding, ar...",total number total agrl land holder total agri...,"[[-0.13323000073432922, -0.3314499855041504, -..."
1163,1170.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Male,Area,Total Agrl. Land Holder Total, Agric...","[male, area, agrl, land, holder, agricultural,...",male area total agrl land holder total agricul...,"[[-0.324180006980896, 0.4799000024795532, -0.1..."
1164,1171.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Female,Area,Total Agrl. Land Holder Total, Agr...","[female, area, agrl, land, holder, agricultura...",female area total agrl land holder total agric...,"[[-0.22990000247955322, 0.4291200041770935, -0..."
1165,1172.0,18,Additional information,18.1,Agricultural Land Holdings And Area per 2010-1...,"Institutions,Area,Total Agrl. Land Holder Tota...","[institution, area, agrl, land, holder, agricu...",institutions area total agrl land holder total...,"[[-0.035638000816106796, -0.29197999835014343,..."


In [141]:
missing_words_set = list(set(missing_words))
print(len(missing_words_set), missing_words_set)

76 [('KAG', 'ariculated'), ('KAG', 'bhagyajyoti'), ('KAG', 'yojane'), ('KAG', 'intons'), ('KAG', 'nigerseed'), ('KAG', 'allopathy'), ('KAG', 'bhagyalakshmi'), ('KAG', 'ipsets'), ('KAG', 'rikshaws'), ('KAG', 'anganawadi'), ('KAG', 'kscard'), ('KAG', 'proporationof'), ('KAG', 'avare'), ('KAG', 'creeches'), ('KAG', 'mathsasraya'), ('KAG', 'nitrozen'), ('KAG', 'sterlisation'), ('KAG', 'intonnes'), ('KAG', 'districtwise'), ('KAG', 'raingauge'), ('KAG', 'sapota'), ('KAG', 'sqkms'), ('KAG', 'hoblies'), ('KAG', 'totalworkers'), ('KAG', 'benificiaries'), ('KAG', 'lpcd'), ('KAG', 'handpump'), ('KAG', 'sectorwise'), ('KAG', 'nrlm'), ('KAG', 'ofemployment'), ('KAG', 'nddp'), ('KAG', 'mandays'), ('KAG', 'distrubuted'), ('KAG', 'raods'), ('KAG', 'indigenious'), ('KAG', 'woked'), ('KAG', 'thousandnuts'), ('KAG', 'kutera'), ('KAG', 'bengalgram'), ('KAG', 'referal'), ('KAG', 'famalies'), ('KAG', 'delivaries'), ('KAG', 'notavailable'), ('KAG', 'estp'), ('KAG', 'agricensus'), ('KAG', 'otal'), ('KAG', 'ja

In [142]:
miss_added_set = list(set([item for sublist in miss_added for item in sublist]))
print(len(miss_added_set), miss_added_set)

76 ['ariculated', 'intons', 'totalworkers', 'otal', 'distrubuted', 'guarentee', 'drbrambedkar', 'sesamum', 'bhagyalakshmi', 'sterlisation', 'ashraya', 'hepatities', 'rainfallfrom', 'nigerseed', 'anthyodaya', 'allopathy', 'benificiaries', 'mahathma', 'facilitiesin', 'anganawadi', 'famalies', 'handpump', 'ofemployment', 'referal', 'kscard', 'thousandnuts', 'intonnes', 'delivaries', 'awnganawadi', 'tubewell', 'yojane', 'jananni', 'ployment', 'sectorwise', 'notavailable', 'mathsasraya', 'rainydays', 'avare', 'raods', 'mtonnes', 'woked', 'ipsets', 'mgnregs', 'agrl', 'bengalgram', 'rikshaws', 'selfemployment', 'sapota', 'suraksha', 'estp', 'lpcd', 'districtwise', 'bhagyajyoti', 'kutera', 'loantaken', 'rainguage', 'creeches', 'nddp', 'nrlm', 'agricensus', 'hoblies', 'multiaxled', 'pomagranate', 'shakt', 'gddp', 'rsin', 'sqkms', 'raingauge', 'vehical', 'nitrozen', 'mandays', 'tubectomy', 'agril', 'proporationof', 'indigenious', 'omini']


In [143]:
sdg_count_threshold = sdg_sem.copy()

In [144]:
min_val_list, top_n_index_list = find_similarity_semantic(sdg_count_threshold, new_df, 10)

In [150]:
sdg_count_threshold['top_n_index'] = top_n_index_list
sdg_count_threshold['min_values'] = min_val_list
sdg_count_threshold

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,keywords,vec,top_n_index,min_values
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...,"[[poverty, line], [population], [age, group], ...","[[-0.07521600276231766, 0.474590003490448, 0.2...","[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 26, 2...","[0.8120515928906179, 1.0000000000000009, 0.794..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...,"[[living, national], [poverty, line], [age, gr...","[[-0.21916300430893898, 0.3655640622600913, 0....","[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 25, 2...","[0.6348859380215051, 0.8120515928906179, 0.794..."
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...,"[[according, national], [children, ages], [liv...","[[-0.17308800294995308, 0.43849001079797745, 0...","[511, 872, 873, 874, 875, 956, 25, 26, 27, 28,...","[0.5844719405576027, 0.6779166506008225, 0.885..."
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...,"[[age, persons], [children], [population, cove...","[[-0.5623050034046173, 0.039556750562042, 0.20...","[715, 716, 717, 718, 719, 720, 721, 722, 723, ...","[0.8303263734472492, 0.7450329169830372, 0.835..."
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...,"[[basic, services], [population], [living], [a...","[[-0.6532500088214874, -0.4114450067281723, -0...","[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 515, ...","[0.6746278273013793, 1.0000000000000009, 0.586..."
...,...,...,...,...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",Total amount of approved funding to promote th...,16.4,16.4.1,"[amount, approved, funding, promote, developme...",total amount of approved funding to promote th...,"[[development], [sound, technologies], [promot...","[[-0.10136999934911728, -0.27173998951911926, ...","[836, 837, 839, 841, 842, 845, 854, 857, 859, ...","[1.0000000000000009, 0.47943460610968414, 0.56..."
175,16,Strengthen the means of implementation and rev...,Fully operationalize the technology bank and s...,Proportion of individuals using the Internet.,16.5,16.5.1,"[individual, using, internet]",proportion of individuals using the internet,"[[individual], [using], [internet]]","[[-0.5918700098991394, 0.49957001209259033, -0...","[865, 525, 891, 892, 893, 894, 910, 911, 912, ...","[0.5254031344856757, 0.8469151097350804, 0.567..."
176,17,"Data, monitoring and accountability","By 2020, enhance capacity-building support to ...",Proportion of sustainable development indicato...,17.1,17.1.1,"[sustainable, development, indicator, produced...",proportion of sustainable development indicato...,"[[official, statistics], [level], [target, acc...","[[-0.10437500174157321, 0.33179498463869095, 0...","[836, 837, 839, 841, 842, 845, 854, 857, 859, ...","[0.5360062829840535, 0.5584521024536246, 0.456..."
177,17,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...

In [151]:
sdg_min_value = sdg_count_threshold.copy()

In [152]:
sdg_count_threshold_all = finding_attr_details(sdg_count_threshold, new_df)
sdg_count_threshold_all

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,keywords,vec,top_n_index,min_values,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...,"[[poverty, line], [population], [age, group], ...","[[-0.07521600276231766, 0.474590003490448, 0.2...","[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 26, 2...","[0.8120515928906179, 1.0000000000000009, 0.794...",54,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.2, 2.2, 2.2, ...","[Geographical Area Sq.Kms, Population and perc...","[Area and Population, Area and Population, Are...",[Population and percentage share to total Popu...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...,"[[living, national], [poverty, line], [age, gr...","[[-0.21916300430893898, 0.3655640622600913, 0....","[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 25, 2...","[0.6348859380215051, 0.8120515928906179, 0.794...",50,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.2, 2.2, 2.2, ...","[Geographical Area Sq.Kms, Population and perc...","[Area and Population, Area and Population, Are...",[Population and percentage share to total Popu...
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...,"[[according, national], [children, ages], [liv...","[[-0.17308800294995308, 0.43849001079797745, 0...","[511, 872, 873, 874, 875, 956, 25, 26, 27, 28,...","[0.5844719405576027, 0.6779166506008225, 0.885...",57,"[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[9, 14, 14, 14, 14, 17, 1, 1, 1, 1, 1, 1, 1, 1...","[9.2, 14.3, 14.3, 14.3, 14.3, 17.2, 1.6, 1.6, ...","[National Highway,P.w.d. Road Length in Km, P....","[TRANSPORT AND COMMUNICATION, Rural Developmen...","[P.w.d. Road Length on 31-03-2017, Physical Pr..."
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...,"[[age, persons], [children], [population, cove...","[[-0.5623050034046173, 0.039556750562042, 0.20...","[715, 716, 717, 718, 719, 720, 721, 722, 723, ...","[0.8303263734472492, 0.7450329169830372, 0.835...",56,"[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 2, 2,...","[12.1, 12.1, 12.1, 12.1, 12.1, 12.1, 12.1, 12....","[Nos,Boys,Government- Pre-Matric, Scheduled Ca...","[Social Welfare, Social Welfare, Social Welfar...",[Scheduled Caste Student Hostels As on 31-3-20...
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...,"[[basic, services], [population], [living], [a...","[[-0.6532500088214874, -0.4114450067281723, -0...","[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 515, ...","[0.6746278273013793, 1.0000000000000009, 0.586...",57,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 9, 9, 9, 9, 9, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.2, 2.2, 2.2, ...","[Geographi

In [153]:
sdg_copy = sdg_sem.copy()

In [154]:
sdg_count_threshold_all_goals = process_result_semantic(sdg_count_threshold_all, sdg_copy)

In [155]:
sdg_count_threshold_all.to_csv(absolute_path_semantic, sep=";", index=False)

In [156]:
sdg_count_threshold_all_goals.to_csv(absolute_path_semantic_unroll, sep=";", index=False)

In [157]:
sdg_count_threshold_all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,Tentative Indicators,top_n_goal_id,top_n_target_id,top_n_indicator_id
0,58.0,2,Area and Population,2.1,Population and percentage share to total Popul...,"Geographical Area Sq.Kms, Population and perce...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
1,59.0,2,Area and Population,2.1,Population and percentage share to total Popul...,"Total,Population, Population and percentage sh...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
2,60.0,2,Area and Population,2.1,Population and percentage share to total Popul...,"Male,Population, Population and percentage sha...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
3,61.0,2,Area and Population,2.1,Population and percentage share to total Popul...,"Females,Population, Population and percentage ...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
4,62.0,2,Area and Population,2.1,Population and percentage share to total Popul...,"% Share to total Population, Population and pe...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
...,...,...,...,...,...,...,...,...,...,...,...,...
8181,480.0,8,Co-operation and Agricultural Marketing,8.3,Credit Co-operative Societies on 31-3-2017,"Society,Total,Credit Co-operative Societies, C...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
8182,481.0,8,Co-operation and Agricultural Marketing,8.3,Credit Co-operative Societies on 31-3-2017,"Members,Total,Credit Co-operative Societies, C...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
8183,482.0,8,Co-operation and Agricultural Marketing,8.4,Non-Credit Co-operative Societies on 31-3-2017,"Society,Marketing,Non-Credit Co-operative Soci...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
8184,483.0,8,Co-operation and Agricultural Marketing,8.4,Non-Credit Co-operative Societies on 31-3-2017,"Members,Marketing,Non-Credit Co-operative Soci...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2


In [158]:
sdg_count_sim = sdg_sem.copy()

In [159]:
top_index = find_similarity_semantic_threshold(sdg_min_value, new_df, 0.025)

In [160]:
sdg_count_sim['top_n_index'] = top_index

In [161]:
sdg_count_sim_all = finding_attr_details(sdg_count_sim, new_df)
sdg_count_sim_all

,Goal No.,Goal,Targets,Tentative Indicators,Target_id,Indicator_id,new_indicators,new_indicators_str,keywords,vec,top_n_index,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1.1,1.1.1,"[population, international, poverty, line, dis...",proportion of the population below the interna...,"[[poverty, line], [population], [age, group], ...","[[-0.07521600276231766, 0.474590003490448, 0.2...","[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 6...",386,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.2, 2.2, 2.2, ...","[Geographical Area Sq.Kms, Population and perc...","[Area and Population, Area and Population, Are...",[Population and percentage share to total Popu...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",Proportion of the population living below the ...,1.2,1.2.1,"[population, living, national, poverty, line, ...",proportion of the population living below the ...,"[[living, national], [poverty, line], [age, gr...","[[-0.21916300430893898, 0.3655640622600913, 0....","[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 6...",135,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.2, 2.2, 2.2, ...","[Geographical Area Sq.Kms, Population and perc...","[Area and Population, Area and Population, Are...",[Population and percentage share to total Popu...
2,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...","Proportion of men, women and children of all a...",1.2,1.2.2,"[men, woman, child, age, living, poverty, dime...",proportion of men women and children of all ag...,"[[according, national], [children, ages], [liv...","[[-0.17308800294995308, 0.43849001079797745, 0...","[511, 872, 873, 874, 875, 956, 25, 26, 27, 28,...",307,"[519.0, 879.0, 880.0, 881.0, 882.0, 963.0, 28....","[9, 14, 14, 14, 14, 17, 1, 1, 1, 1, 1, 1, 1, 1...","[9.2, 14.3, 14.3, 14.3, 14.3, 17.2, 1.6, 1.6, ...","[National Highway,P.w.d. Road Length in Km, P....","[TRANSPORT AND COMMUNICATION, Rural Developmen...","[P.w.d. Road Length on 31-03-2017, Physical Pr..."
3,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,Percentage of the population covered by social...,1.3,1.3.1,"[population, covered, social, protection, floo...",percentage of the population covered by social...,"[[age, persons], [children], [population, cove...","[[-0.5623050034046173, 0.039556750562042, 0.20...","[715, 716, 717, 718, 719, 720, 721, 722, 723, ...",339,"[722.0, 723.0, 724.0, 725.0, 726.0, 727.0, 728...","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 1...","[12.1, 12.1, 12.1, 12.1, 12.1, 12.1, 12.1, 12....","[Nos,Boys,Government- Pre-Matric, Scheduled Ca...","[Social Welfare, Social Welfare, Social Welfar...",[Scheduled Caste Student Hostels As on 31-3-20...
4,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",Proportion of the population living in househo...,1.4,1.4.1,"[population, living, household, access, basic,...",proportion of the population living in househo...,"[[basic, services], [population], [living], [a...","[[-0.6532500088214874, -0.4114450067281723, -0...","[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 6...",484,"[58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65....","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[2.1, 2.1, 2.1, 2.1, 2.1, 2.1, 2.2, 2.2, 2.2, ...","[Geographical Area Sq.Kms, Population and perc...","[Area and Population, Area and Population, Are...",[Population and percentage share to total Popu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation a

In [162]:
sdg_copy2 = sdg_sem.copy()

In [163]:
sdg_count_sim_all_goals = process_result_semantic(sdg_count_sim_all, sdg_copy2)

In [164]:
sdg_count_sim_all.to_csv(absolute_path_sem_threshold, sep=";", index=False)

In [165]:
sdg_count_sim_all_goals.to_csv(absolute_path_sem_threshold_unroll, sep=";", index=False)

In [166]:
sdg_count_sim_all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,Tentative Indicators,top_n_goal_id,top_n_target_id,top_n_indicator_id
0,58.0,2,Area and Population,2.1,Population and percentage share to total Popul...,"Geographical Area Sq.Kms, Population and perce...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
1,59.0,2,Area and Population,2.1,Population and percentage share to total Popul...,"Total,Population, Population and percentage sh...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
2,60.0,2,Area and Population,2.1,Population and percentage share to total Popul...,"Male,Population, Population and percentage sha...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
3,61.0,2,Area and Population,2.1,Population and percentage share to total Popul...,"Females,Population, Population and percentage ...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
4,62.0,2,Area and Population,2.1,Population and percentage share to total Popul...,"% Share to total Population, Population and pe...",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",Proportion of the population below the interna...,1,1.1,1.1.1
...,...,...,...,...,...,...,...,...,...,...,...,...
48306,672.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Nos.,Primary Health Centres, Allopathy Hospita...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
48307,673.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Beds,Primary Health Centres, Allopathy Hospita...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
48308,674.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Nos.,Community Health Centres, Allopathy Hospi...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
48309,675.0,11,Health & Family welfare Services,11.3,Allopathy Hospitals Indian System No.of Medici...,"Beds,Community Health Centres, Allopathy Hospi...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",Inclusive Wealth Index.,17,17.2,17.2.2
